# RAIL Evaluation - Check results against DC1 paper

Contact: _Julia Gschwend_ ([julia@linea.gov.br](mailto:julia@linea.gov.br)), _Sam Schmidt, Alex Malz, Eric Charles_

The purpose of this notebook is to validate the new implementation of the DC1 metrics, previously available on Github repository [PZDC1paper](https://github.com/LSSTDESC/PZDC1paper), now refactored to be part of RAIL Evaluation module. The metrics here were implemented in object-oriented Python 3, inheriting from qp.Ensamble. In this notebook we use the same input dataset used in DC1 PZ paper ([Schmidt et al. 2020](https://arxiv.org/pdf/2001.03621.pdf)), copied from cori (/global/cfs/cdirs/lsst/groups/PZ/PhotoZDC1/photoz_results/TESTDC1FLEXZ) to reproduce its results.

In [ ]:
from IPython.display import Markdown
import os
from sample import Sample
from metrics import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

<a class="anchor" id="sample"></a>

## Sample  



In [ ]:
my_path = "/Users/julia/TESTDC1FLEXZ"

pdfs_file =  os.path.join(my_path, "Mar5Flexzgold_pz.out")
ztrue_file =  os.path.join(my_path, "Mar5Flexzgold_idszmag.out")

#pdfs_file =  os.path.join(my_path, "1pct_Mar5Flexzgold_pz.out")
#ztrue_file =  os.path.join(my_path, "1pct_Mar5Flexzgold_idszmag.out")

In [ ]:
sample = Sample(pdfs_file, ztrue_file, code="FZBoost", name="DC1 paper data")
sample

In [ ]:
print(sample)

In [ ]:
sample.plot_old_valid()#gals=gals, colors=colors)

## Metrics

In [ ]:
%%time
metrics = Metrics(sample)

#### PIT-QQ plot

In [ ]:
metrics.plot_pit_qq() #savefig=True)

Metrics and DC1 paper results for comparison:

In [ ]:
metrics.markdown_summary_metrics(show_dc1=True)

## Conclusion

<font color=red> TBD </font>